In [5]:
import pandas as pd

In [6]:
#caminho Dani
bruto = pd.read_csv(r'C:\JupyterLab\2° Semestre\Machine Learning\ML-2022-DIMP\Neutral_Graphene_Oxide_Nanoflake_Dataset.csv')

In [ ]:
#caminho Isa
bruto = pd.read_csv()

In [ ]:
#caminho Marcos
bruto = pd.read_csv(r'C:\Users\marcos220057\OneDrive - CNPEM - Centro Nacional de Pesquisa em Energia e Materiais\2sem2022\Aprendizado de Máquina\Neutral_Graphene_Oxide_Nanoflake_Dataset.csv')

In [9]:
#caminho Paola
bruto = pd.read_csv(r'C:\JupyterLab\ML-2022-DIMP\Neutral_Graphene_Oxide_Nanoflake_Dataset.csv')

In [18]:
# Excluir as colunas que tem NaN
bruto_completo = bruto.dropna(axis = 1, how ='any')

In [19]:
#Calculado a matriz de correlação peterson para ver quais atributos são bons para energia de Fermi
bruto_completo.shape
corrmat = bruto_completo.corr()

In [28]:
#printando os atributos que correlacionam melhor que 0.7
for i in corrmat['Fermi_energy'].keys():
    if abs(corrmat['Fermi_energy'][i]) > 0.782:
        print(i, 'correlation:', corrmat['Fermi_energy'][i])
        print()

O_concentration correlation: -0.7963203388750189

C-C-C_C-sp2-C:error correlation: -0.8493752886340112

C-O_sp3-sp1:mean_value correlation: 0.847305780443348

C-O_sp3-strained:mean_value correlation: 0.8280893147336332

C-O-H:mean_value correlation: -0.8274333876940453

C-O-H_C-sp1-H:mean_value correlation: -0.8273869890961334

C-O_coordination_number correlation: -0.7868925300771553

density_loc_ether_mean correlation: -0.783175035923612

norm_density_ether_mean correlation: -0.7869970791019995

norm_Izz_bar_ether correlation: -0.7841549332099131

Fermi_energy correlation: 1.0



In [36]:
for i in corrmat['total_energy'].keys():
    if abs(corrmat['total_energy'][i]) > 0.93:
        print(i, corrmat['total_energy'][i])
        print()

C -0.9451088774529183

H -0.9415750852322793

atom_number_total -0.997881550895204

area -0.946461806575993

all_agent_group_count -0.9546818573050628

C-C:total_number -0.9442555104184308

C-C_sp2-sp3:total_number -0.9376195769770793

C-C-C:total_number -0.9432530823044715

rings_total_number_10_all -0.9333293400696462

rings_total_number_6_all -0.9377238761263992

rings_total_number_6_sp -0.9375247370495672

Ixx_ether -0.9343988890313029

Izz_ether -0.9393554727019133

norm_Ixx_ether -0.9310057193670633

norm_Izz_ether -0.9354374049002103

total_energy 1.0

